In [15]:
import pandas as pd
import numpy as np

In [16]:
ebp_url = 'https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/electrification_by_province.csv'
ebp_df = pd.read_csv(ebp_url)

for col, row in ebp_df.iloc[:,1:].iteritems():
    ebp_df[col] = ebp_df[col].str.replace(',','').astype(int)

ebp_df.head()

,Financial Year (1 April - 30 March),Limpopo,Mpumalanga,North west,Free State,Kwazulu Natal,Eastern Cape,Western Cape,Northern Cape,Gauteng
0,2000/1,51860,28365,48429,21293,63413,49008,48429,6168,39660
1,2001/2,68121,26303,38685,20928,64123,45773,38685,10359,36024
2,2002/3,49881,11976,28532,10316,63078,55748,28532,6869,32127
3,2003/4,42034,33515,34027,16135,60282,47414,34027,10976,39488
4,2004/5,54646,16218,21450,5668,37811,42041,21450,6316,18422


In [17]:
twitter_url = 'https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/twitter_nov_2019.csv'
twitter_df = pd.read_csv(twitter_url)
twitter_df.head()

,Tweets,Date
0,@BongaDlulane Please send an email to mediades...,2019-11-29 12:50:54
1,@saucy_mamiie Pls log a call on 0860037566,2019-11-29 12:46:53
2,@BongaDlulane Query escalated to media desk.,2019-11-29 12:46:10
3,"Before leaving the office this afternoon, head...",2019-11-29 12:33:36
4,#ESKOMFREESTATE #MEDIASTATEMENT : ESKOM SUSPEN...,2019-11-29 12:17:43


In [19]:
# gauteng ebp data as a list
gauteng = ebp_df['Gauteng'].astype(float).to_list()

# dates for twitter tweets
dates = twitter_df['Date'].to_list()

# dictionary mapping official municipality twitter handles to the municipality name
mun_dict = {
    '@CityofCTAlerts' : 'Cape Town',
    '@CityPowerJhb' : 'Johannesburg',
    '@eThekwiniM' : 'eThekwini' ,
    '@EMMInfo' : 'Ekurhuleni',
    '@centlecutility' : 'Mangaung',
    '@NMBmunicipality' : 'Nelson Mandela Bay',
    '@CityTshwane' : 'Tshwane'
}

# dictionary of english stopwords
stop_words_dict = {
    'stopwords':[
        'where', 'done', 'if', 'before', 'll', 'very', 'keep', 'something', 'nothing', 'thereupon', 
        'may', 'why', 'â€™s', 'therefore', 'you', 'with', 'towards', 'make', 'really', 'few', 'former', 
        'during', 'mine', 'do', 'would', 'of', 'off', 'six', 'yourself', 'becoming', 'through', 
        'seeming', 'hence', 'us', 'anywhere', 'regarding', 'whole', 'down', 'seem', 'whereas', 'to', 
        'their', 'various', 'thereafter', 'â€˜d', 'above', 'put', 'sometime', 'moreover', 'whoever', 'although', 
        'at', 'four', 'each', 'among', 'whatever', 'any', 'anyhow', 'herein', 'become', 'last', 'between', 'still', 
        'was', 'almost', 'twelve', 'used', 'who', 'go', 'not', 'enough', 'well', 'â€™ve', 'might', 'see', 'whose', 
        'everywhere', 'yourselves', 'across', 'myself', 'further', 'did', 'then', 'is', 'except', 'up', 'take', 
        'became', 'however', 'many', 'thence', 'onto', 'â€˜m', 'my', 'own', 'must', 'wherein', 'elsewhere', 'behind', 
        'becomes', 'alone', 'due', 'being', 'neither', 'a', 'over', 'beside', 'fifteen', 'meanwhile', 'upon', 'next', 
        'forty', 'what', 'less', 'and', 'please', 'toward', 'about', 'below', 'hereafter', 'whether', 'yet', 'nor', 
        'against', 'whereupon', 'top', 'first', 'three', 'show', 'per', 'five', 'two', 'ourselves', 'whenever', 
        'get', 'thereby', 'noone', 'had', 'now', 'everyone', 'everything', 'nowhere', 'ca', 'though', 'least', 
        'so', 'both', 'otherwise', 'whereby', 'unless', 'somewhere', 'give', 'formerly', 'â€™d', 'under', 
        'while', 'empty', 'doing', 'besides', 'thus', 'this', 'anyone', 'its', 'after', 'bottom', 'call', 
        'nâ€™t', 'name', 'even', 'eleven', 'by', 'from', 'when', 'or', 'anyway', 'how', 'the', 'all', 
        'much', 'another', 'since', 'hundred', 'serious', 'â€˜ve', 'ever', 'out', 'full', 'themselves', 
        'been', 'in', "'d", 'wherever', 'part', 'someone', 'therein', 'can', 'seemed', 'hereby', 'others', 
        "'s", "'re", 'most', 'one', "n't", 'into', 'some', 'will', 'these', 'twenty', 'here', 'as', 'nobody', 
        'also', 'along', 'than', 'anything', 'he', 'there', 'does', 'we', 'â€™ll', 'latterly', 'are', 'ten', 
        'hers', 'should', 'they', 'â€˜s', 'either', 'am', 'be', 'perhaps', 'â€™re', 'only', 'namely', 'sixty', 
        'made', "'m", 'always', 'those', 'have', 'again', 'her', 'once', 'ours', 'herself', 'else', 'has', 'nine', 
        'more', 'sometimes', 'your', 'yours', 'that', 'around', 'his', 'indeed', 'mostly', 'cannot', 'â€˜ll', 'too', 
        'seems', 'â€™m', 'himself', 'latter', 'whither', 'amount', 'other', 'nevertheless', 'whom', 'for', 'somehow', 
        'beforehand', 'just', 'an', 'beyond', 'amongst', 'none', "'ve", 'say', 'via', 'but', 'often', 're', 'our', 
        'because', 'rather', 'using', 'without', 'throughout', 'on', 'she', 'never', 'eight', 'no', 'hereupon', 
        'them', 'whereafter', 'quite', 'which', 'move', 'thru', 'until', 'afterwards', 'fifty', 'i', 'itself', 'nâ€˜t',
        'him', 'could', 'front', 'within', 'â€˜re', 'back', 'such', 'already', 'several', 'side', 'whence', 'me', 
        'same', 'were', 'it', 'every', 'third', 'together'
    ]
}

In [20]:
twitter_df

,Tweets,Date
0,@BongaDlulane Please send an email to mediades...,2019-11-29 12:50:54
1,@saucy_mamiie Pls log a call on 0860037566,2019-11-29 12:46:53
2,@BongaDlulane Query escalated to media desk.,2019-11-29 12:46:10
3,"Before leaving the office this afternoon, head...",2019-11-29 12:33:36
4,#ESKOMFREESTATE #MEDIASTATEMENT : ESKOM SUSPEN...,2019-11-29 12:17:43
...,...,...
195,Eskom's Visitors Centres’ facilities include i...,2019-11-20 10:29:07
196,#Eskom connected 400 houses and in the process...,2019-11-20 10:25:20
197,@ArthurGodbeer Is the power restored as yet?,2019-11-20 10:07:59
198,@MuthambiPaulina @SABCNewsOnline @IOL @eNCA @e...,2019-11-20 10:07:41


In [21]:
twitter_df['Tweets'] = twitter_df['Tweets'].replace({'@':''}, regex=True)

In [22]:
twitter_df['Tweets']

0      BongaDlulane Please send an email to mediadesk...
1              saucy_mamiie Pls log a call on 0860037566
2            BongaDlulane Query escalated to media desk.
3      Before leaving the office this afternoon, head...
4      #ESKOMFREESTATE #MEDIASTATEMENT : ESKOM SUSPEN...
                             ...                        
195    Eskom's Visitors Centres’ facilities include i...
196    #Eskom connected 400 houses and in the process...
197          ArthurGodbeer Is the power restored as yet?
198    MuthambiPaulina SABCNewsOnline IOL eNCA ewnupd...
199    RT GP_DHS: The GautengProvince made a commitme...
Name: Tweets, Length: 200, dtype: object

In [23]:
### START FUNCTION
def extract_municipality_hashtags(df):
    """ Extracts the municipality and hashtags from tweets. Adds extracted data to new columns
    
    Parameters
    ----------
    df: Pandas dataframe
       The dataframe which muncipality and hashtag will be added to.
       
    Returns
    -------
    df: Pandas dataFrame
          Returns data in a column named "hashtags" and a column named "municipality"
    """
    
    split_list=[]  #initiating empty lists
    h_list=[]
    mun_list=[]

    for i in df["Tweets"]:                      #run loop through tweets column
        
        split_list=i.split()                    #split each tweet
        flag1=False                             #creating a boolean variable
        in_list=[]                              #creating a blank inner list
        
        for a in split_list:                    #run loop through split tweet list
            if a.startswith("#"):               #extracting words that begin with a hashtag
                in_list.append(a.lower())       #making hashtag words to lowercase and appending it 
                flag1=True                      #setting flag to true
                
        if flag1 == True:                       #checking flag condition
            h_list.append(in_list)              #append inner list to hashtag list
        else:
            h_list.append(np.nan)               #append a NuN
            
        flag2=False                             #declaring a new boolean variable 
        for key in mun_dict:                    #looping through keys in dictionary
            if key in i:                        #finding if key is in tweet
                mun_list.append(mun_dict[key])  #appending value of key to municpality list
                flag2 = True                    #setting flag to true if key is found
                
        if flag2==True:                         #checking flag2 condition
            pass
        else:
            mun_list.append(np.nan)             #append NuN if flag condition is not met
            
    df["municipality"] = mun_list               #creating municipality column in df
    df["hashtags"] = h_list                     #creating hashtag column in df
    
    return df

### END FUNCTION

In [24]:
mun_dict = {
    '@CityofCTAlerts' : 'Cape Town',
    '@CityPowerJhb' : 'Johannesburg',
    '@eThekwiniM' : 'eThekwini' ,
    '@EMMInfo' : 'Ekurhuleni',
    '@centlecutility' : 'Mangaung',
    '@NMBmunicipality' : 'Nelson Mandela Bay',
    '@CityTshwane' : 'Tshwane'
}

for key in mun_dict:
    if key != key.replace('@', ''):
        mun_dict[key.replace('@', '')] = mun_dict[key]
        del mun_dict[key]

print(mun_dict)

{'@centlecutility': 'Mangaung', '@NMBmunicipality': 'Nelson Mandela Bay', '@CityTshwane': 'Tshwane', 'CityofCTAlerts': 'Cape Town', 'CityPowerJhb': 'Johannesburg', 'eThekwiniM': 'eThekwini', 'EMMInfo': 'Ekurhuleni'}


In [25]:
mun_dict

{'@centlecutility': 'Mangaung',
 '@NMBmunicipality': 'Nelson Mandela Bay',
 '@CityTshwane': 'Tshwane',
 'CityofCTAlerts': 'Cape Town',
 'CityPowerJhb': 'Johannesburg',
 'eThekwiniM': 'eThekwini',
 'EMMInfo': 'Ekurhuleni'}

In [26]:
mun_dict

{'@centlecutility': 'Mangaung',
 '@NMBmunicipality': 'Nelson Mandela Bay',
 '@CityTshwane': 'Tshwane',
 'CityofCTAlerts': 'Cape Town',
 'CityPowerJhb': 'Johannesburg',
 'eThekwiniM': 'eThekwini',
 'EMMInfo': 'Ekurhuleni'}

In [27]:
mun_dict1 = {
    '@centlecutility' : 'Mangaung',
    '@NMBmunicipality' : 'Nelson Mandela Bay',
    '@CityTshwane' : 'Tshwane'
}

for key in mun_dict:
    if key != key.replace('@', ''):
        mun_dict[key.replace('@', '')] = mun_dict[key]
        del mun_dict[key]

print(mun_dict)

{'CityofCTAlerts': 'Cape Town', 'CityPowerJhb': 'Johannesburg', 'eThekwiniM': 'eThekwini', 'EMMInfo': 'Ekurhuleni', 'centlecutility': 'Mangaung', 'NMBmunicipality': 'Nelson Mandela Bay', 'CityTshwane': 'Tshwane'}


In [28]:
mun_dict

{'CityofCTAlerts': 'Cape Town',
 'CityPowerJhb': 'Johannesburg',
 'eThekwiniM': 'eThekwini',
 'EMMInfo': 'Ekurhuleni',
 'centlecutility': 'Mangaung',
 'NMBmunicipality': 'Nelson Mandela Bay',
 'CityTshwane': 'Tshwane'}

In [29]:
d = {'A1':2345, 'A2':23,'A3':1}
d['A1'] = int(repr(d['A1'])[2:]) # getting slice of characters starting from index 2

print(d)

{'A1': 45, 'A2': 23, 'A3': 1}
